In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import scipy as sp

In [22]:
from jupyter_dash import JupyterDash
import dash
from dash import dcc
from dash import html
import pandas as pd




df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')
available_indicators = df['Indicator Name'].unique()


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets = external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

#######################################################################################
app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Fertility rate, total (births per woman)'
            ),
            dcc.RadioItems(
                id='crossfilter-xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                id='crossfilter-yaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Life expectancy at birth, total (years)'
            ),
            dcc.RadioItems(
                id='crossfilter-yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'
    }),

    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': 'Japan'}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),

    html.Div(dcc.Slider(
        id='crossfilter-year--slider',
        min=df['Year'].min(),
        max=df['Year'].max(),
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()},
        step=None
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])

#####################################################################################



@app.callback(
    dash.dependencies.Output('crossfilter-indicator-scatter', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-year--slider', 'value')])
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]

    return {
        'data': [dict(
            x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
            y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
            text=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'],
            customdata=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'],
            mode='markers',
            marker={
                'size': 25,
                'opacity': 0.7,
                'color': 'orange',
                'line': {'width': 2, 'color': 'purple'}
            }
        )],
        'layout': dict(
            xaxis={
                'title': xaxis_column_name,
                'type': 'linear' if xaxis_type == 'Linear' else 'log'
            },
            yaxis={
                'title': yaxis_column_name,
                'type': 'linear' if yaxis_type == 'Linear' else 'log'
            },
            margin={'l': 40, 'b': 30, 't': 10, 'r': 0},
            height=450,
            hovermode='closest'
        )
    }


def create_time_series(dff, axis_type, title):
    return {
        'data': [dict(
            x=dff['Year'],
            y=dff['Value'],
            mode='lines+markers'
        )],
        'layout': {
            'height': 225,
            'margin': {'l': 20, 'b': 30, 'r': 10, 't': 10},
            'annotations': [{
                'x': 0, 'y': 0.85, 'xanchor': 'left', 'yanchor': 'bottom',
                'xref': 'paper', 'yref': 'paper', 'showarrow': False,
                'align': 'left', 'bgcolor': 'rgba(255, 255, 255, 0.5)',
                'text': title
            }],
            'yaxis': {'type': 'linear' if axis_type == 'Linear' else 'log'},
            'xaxis': {'showgrid': False}
        }
    }


@app.callback(
    dash.dependencies.Output('x-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value')])
def update_y_timeseries(hoverData, xaxis_column_name, axis_type):
    country_name = hoverData['points'][0]['customdata']
    dff = df[df['Country Name'] == country_name]
    dff = dff[dff['Indicator Name'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)


@app.callback(
    dash.dependencies.Output('y-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value')])
def update_x_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['Country Name'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['Indicator Name'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)


#app.run_server(mode="external")
app.run_server(mode="inline", host="localhost",port=8051)

In [20]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
# Load Data
df = px.data.tips()
print(df.head())
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("JupyterDash Demo"),
    dcc.Graph(id='graph'),
    html.Label([
        "colorscale",
        dcc.Dropdown(
            id='colorscale-dropdown', clearable=False,
            value='plasma', options=[
                {'label': c, 'value': c}
                for c in px.colors.named_colorscales()
            ])
    ]),
])

# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("colorscale-dropdown", "value")]
)
def update_figure(colorscale):
    return px.scatter(
        df, x="total_bill", y="tip", color="size",
        color_continuous_scale=colorscale,
        render_mode="webgl", title="Tips"
    )
# Run app and display result inline in the notebook
app.run_server(mode='inline', host="localhost",port=8051)

C:\Users\SRGIO~1\AppData\Local\Temp/ipykernel_5028/2308646719.py:3: UserWarning:


The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`

C:\Users\SRGIO~1\AppData\Local\Temp/ipykernel_5028/2308646719.py:4: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`



   total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2
4       24.59  3.61  Female     No  Sun  Dinner     4


In [21]:
from dash import Dash, html, dcc

import plotly.express as px
import pandas as pd

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

app = JupyterDash(__name__)

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options



##########################################################################################
df = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
    "Amount": [4, 1, 2, 2, 4, 5],
    "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
})
print(df.head())
fig = px.bar(df, x="Fruit", y="Amount", color="City", barmode="group")
##########################################################################################

##  The layout is composed of a tree of "components" such as html.Div and dcc.Graph.

app.layout = html.Div(children=[
    html.H1(children='My first test with dash'),

    html.Div(children='''
        Dash: A web application framework for your data.
    '''),

    dcc.Graph(
        id='example-graph',
        figure=fig
    )
])
############################################################################################



if __name__ == '__main__':
    app.run_server(mode='inline', host="localhost",port=8051)


     Fruit  Amount      City
0   Apples       4        SF
1  Oranges       1        SF
2  Bananas       2        SF
3   Apples       2  Montreal
4  Oranges       4  Montreal


---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
~\AppData\Roaming\Python\Python39\site-packages\dash\dash.py in dispatch(
    self=<jupyter_dash.jupyter_app.JupyterDash object>
)
   1421         try:
-> 1422             cb = self.callback_map[output]
        cb = undefined
        self.callback_map = {}
        output = 'x-time-series.figure'
   1423             func = cb["callback"]

KeyError: 'x-time-series.figure'

The above exception was the direct cause of the following exception:

KeyError                                  Traceback (most recent call last)
C:\Users\anaconda3\lib\site-packages\flask\app.py in full_dispatch_request(self=<Flask '__main__'>)
   1948             rv = self.preprocess_request()
   1949             if rv is None:
-> 1950                 rv = self.dispatch_request()
        rv = None
        self.dispatch_request = <bound method Flask.dispatch_request of

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
~\AppData\Roaming\Python\Python39\site-packages\dash\dash.py in dispatch(
    self=<jupyter_dash.jupyter_app.JupyterDash object>
)
   1421         try:
-> 1422             cb = self.callback_map[output]
        cb = undefined
        self.callback_map = {}
        output = 'x-time-series.figure'
   1423             func = cb["callback"]

KeyError: 'x-time-series.figure'

The above exception was the direct cause of the following exception:

KeyError                                  Traceback (most recent call last)
C:\Users\anaconda3\lib\site-packages\flask\app.py in full_dispatch_request(self=<Flask '__main__'>)
   1948             rv = self.preprocess_request()
   1949             if rv is None:
-> 1950                 rv = self.dispatch_request()
        rv = None
        self.dispatch_request = <bound method Flask.dispatch_request of

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
~\AppData\Roaming\Python\Python39\site-packages\dash\dash.py in dispatch(
    self=<jupyter_dash.jupyter_app.JupyterDash object>
)
   1421         try:
-> 1422             cb = self.callback_map[output]
        cb = undefined
        self.callback_map = {}
        output = 'x-time-series.figure'
   1423             func = cb["callback"]

KeyError: 'x-time-series.figure'

The above exception was the direct cause of the following exception:

KeyError                                  Traceback (most recent call last)
C:\Users\anaconda3\lib\site-packages\flask\app.py in full_dispatch_request(self=<Flask '__main__'>)
   1948             rv = self.preprocess_request()
   1949             if rv is None:
-> 1950                 rv = self.dispatch_request()
        rv = None
        self.dispatch_request = <bound method Flask.dispatch_request of

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
~\AppData\Roaming\Python\Python39\site-packages\dash\dash.py in dispatch(
    self=<jupyter_dash.jupyter_app.JupyterDash object>
)
   1421         try:
-> 1422             cb = self.callback_map[output]
        cb = undefined
        self.callback_map = {}
        output = 'x-time-series.figure'
   1423             func = cb["callback"]

KeyError: 'x-time-series.figure'

The above exception was the direct cause of the following exception:

KeyError                                  Traceback (most recent call last)
C:\Users\anaconda3\lib\site-packages\flask\app.py in full_dispatch_request(self=<Flask '__main__'>)
   1948             rv = self.preprocess_request()
   1949             if rv is None:
-> 1950                 rv = self.dispatch_request()
        rv = None
        self.dispatch_request = <bound method Flask.dispatch_request of

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
~\AppData\Roaming\Python\Python39\site-packages\dash\dash.py in dispatch(
    self=<jupyter_dash.jupyter_app.JupyterDash object>
)
   1421         try:
-> 1422             cb = self.callback_map[output]
        cb = undefined
        self.callback_map = {}
        output = 'x-time-series.figure'
   1423             func = cb["callback"]

KeyError: 'x-time-series.figure'

The above exception was the direct cause of the following exception:

KeyError                                  Traceback (most recent call last)
C:\Users\anaconda3\lib\site-packages\flask\app.py in full_dispatch_request(self=<Flask '__main__'>)
   1948             rv = self.preprocess_request()
   1949             if rv is None:
-> 1950                 rv = self.dispatch_request()
        rv = None
        self.dispatch_request = <bound method Flask.dispatch_request of


---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
~\AppData\Roaming\Python\Python39\site-packages\dash\dash.py in dispatch(
    self=<jupyter_dash.jupyter_app.JupyterDash object>
)
   1421         try:
-> 1422             cb = self.callback_map[output]
        cb = undefined
        self.callback_map = {}
        output = 'y-time-series.figure'
   1423             func = cb["callback"]

KeyError: 'y-time-series.figure'

The above exception was the direct cause of the following exception:

KeyError                                  Traceback (most recent call last)
C:\Users\anaconda3\lib\site-packages\flask\app.py in full_dispatch_request(self=<Flask '__main__'>)
   1948             rv = self.preprocess_request()
   1949             if rv is None:
-> 1950                 rv = self.dispatch_request()
        rv = None
        self.dispatch_request = <bound method Flask.dispatch_request o

In [5]:
from dash import Dash, html, dcc

import plotly.express as px
import pandas as pd

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

app = JupyterDash(__name__)

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
df = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
    "Amount": [4, 1, 2, 2, 4, 5],
    "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
})

fig = px.bar(df, x="Fruit", y="Amount", color="City", barmode="group")

fig.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)

app.layout = html.Div(style={'backgroundColor': colors['background']}, 
                      children=[html.H1('Hello Dash', style={'textAlign': 'center', 'color': '#7FDBFF'}),

    html.Div(children='Dash: A web application framework for your data.', style={
        'textAlign': 'center',
        'color': colors['text']
    }),

    dcc.Graph(
        id='example-graph-2',
        figure=fig
    )
])

if __name__ == '__main__':
    app.run_server(debug=True, mode='inline', host="localhost",port=8051)

In [6]:
from dash import Dash, html, dcc

import plotly.express as px
import pandas as pd

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

df = pd.read_csv('https://gist.githubusercontent.com/chriddyp/c78bf172206ce24f77d6363a2d754b59/raw/c353e8ef842413cae56ae3920b8fd78468aa4cb2/usa-agricultural-exports-2011.csv')


def generate_table(dataframe, max_rows=10):
    return html.Table([
        html.Thead(
            html.Tr([html.Th(col) for col in dataframe.columns])
        ),
        html.Tbody([
            html.Tr([
                html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
            ]) for i in range(min(len(dataframe), max_rows))
        ])
    ])

##############################################################################
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H4(children='US Agriculture Exports (2011)'),
    generate_table(df)
])
#############################################################################
if __name__ == '__main__':
    app.run_server(debug=True, mode='inline', host="localhost",port=8051)

In [7]:
from dash import Dash, dcc, html
import plotly.express as px
import pandas as pd


import plotly.express as px
import pandas as pd

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output


app = JupyterDash(__name__)

df = pd.read_csv('https://gist.githubusercontent.com/chriddyp/5d1ea79569ed194d432e56108a04d188/raw/a9f9e8076b837d541398e999dcbac2b2826a81f8/gdp-life-exp-2007.csv')

fig = px.scatter(df, x="gdp per capita", y="life expectancy",
                 size="population", color="continent", hover_name="country",
                 log_x=True, size_max=60)

app.layout = html.Div([
    dcc.Graph(
        id='life-exp-vs-gdp',
        figure=fig
    )
])

if __name__ == '__main__':
    app.run_server(debug=True, mode='inline', host="localhost",port=8051)

In [8]:
# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.

from dash import Dash, dcc, html
import plotly.express as px
import pandas as pd


import plotly.express as px
import pandas as pd

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output


app = JupyterDash(__name__)

markdown_text = '''
### Dash and Markdown

Dash apps can be written in Markdown.
Dash uses the [CommonMark](http://commonmark.org/)
specification of Markdown.
Check out their [60 Second Markdown Tutorial](http://commonmark.org/help/)
if this is your first introduction to Markdown!
'''

app.layout = html.Div([
    dcc.Markdown(children=markdown_text)
])

if __name__ == '__main__':
    app.run_server(debug=True, mode='inline', host="localhost",port=8051)

In [9]:
from dash import Dash, dcc, html
import plotly.express as px
import pandas as pd

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output


app = JupyterDash(__name__)

app.layout = html.Div([
#######################################################################################    
    html.Div(children=[
        html.Label('Select Dropdown'),
        dcc.Dropdown(['New York City', 'Montréal', 'San Francisco'], 'Montréal'),
#######################################################################################
        html.Br(),
        html.Label('Multi-Select Dropdown'),
        dcc.Dropdown(['New York City', 'Montréal', 'San Francisco'],
                     ['Montréal', 'San Francisco'],
                     multi=True),
#######################################################################################
        html.Br(),
        html.Label('Radio Items'),
        dcc.RadioItems(['New York City', 'Montréal', 'San Francisco'], 'Montréal'),
    ], style={'padding': 10, 'flex': 1}),
#######################################################################################
    html.Div(children=[
        html.Label('Checkboxes'),
        dcc.Checklist(['New York City', 'Montréal', 'San Francisco'],
                      ['Montréal', 'San Francisco']
        ),
#######################################################################################
        html.Br(),
        html.Label('Text Input'),
        dcc.Input(value='MTL', type='text'),
#######################################################################################
        html.Br(),
        html.Label('Slider'),
        dcc.Slider(
            min=0,
            max=9,
            marks={i: f'Label {i}' if i == 1 else str(i) for i in range(1, 10)},
            value=5,
        ),
    ], style={'padding': 10, 'flex': 1})
], style={'display': 'flex', 'flex-direction': 'row'})

if __name__ == '__main__':
    app.run_server(debug=True, mode='inline', host="localhost",port=8051)

In [10]:
from dash import Dash, dcc, html, Input, Output
from jupyter_dash import JupyterDash



app = JupyterDash(__name__)

app.layout = html.Div([
    html.H6("Change the value in the text box to see callbacks in action!"),
    html.Div([
        "Input: ",
        dcc.Input(id='my-input', value='initial value', type='text')
    ]),
    html.Br(),
    html.Div(id='my-output'),

])


@app.callback(
    Output(component_id='my-output', component_property='children'),
    Input(component_id='my-input', component_property='value')
)
def update_output_div(input_value):
    return f'Output: {input_value}'


if __name__ == '__main__':
    app.run_server(debug=True, mode='inline', host="localhost",port=8051)

In [11]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash

import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv')

print(df.head())

app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Graph(id='graph-with-slider'),
    dcc.Slider(
        df['year'].min(),
        df['year'].max(),
        step=None,
        value=df['year'].min(),
        marks={str(year): str(year) for year in df['year'].unique()},
        id='year-slider'
    )
])


@app.callback(
    Output('graph-with-slider', 'figure'),
    Input('year-slider', 'value'))
def update_figure(selected_year):
    filtered_df = df[df.year == selected_year]

    fig = px.scatter(filtered_df, x="gdpPercap", y="lifeExp",
                     size="pop", color="continent", hover_name="country",
                     log_x=True, size_max=55)

    fig.update_layout(transition_duration=500)

    return fig


if __name__ == '__main__':
    app.run_server(debug=True, mode='inline', host="localhost",port=8051)

       country  year         pop continent  lifeExp   gdpPercap
0  Afghanistan  1952   8425333.0      Asia   28.801  779.445314
1  Afghanistan  1957   9240934.0      Asia   30.332  820.853030
2  Afghanistan  1962  10267083.0      Asia   31.997  853.100710
3  Afghanistan  1967  11537966.0      Asia   34.020  836.197138
4  Afghanistan  1972  13079460.0      Asia   36.088  739.981106


In [20]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

import pandas as pd

from jupyter_dash import JupyterDash


app = JupyterDash(__name__)

df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')


app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Fertility rate, total (births per woman)',
                id='xaxis-column'
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='xaxis-type',
                inline=True
            )
        ], style={'width': '48%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Life expectancy at birth, total (years)',
                id='yaxis-column'
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='yaxis-type',
                inline=True
            )
        ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
    ]),

    dcc.Graph(id='indicator-graphic'),

    dcc.Slider(
        df['Year'].min(),
        df['Year'].max(),
        step=None,
        id='year--slider',
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()},

    )
])


@app.callback(
    Output('indicator-graphic', 'figure'),
    Input('xaxis-column', 'value'),
    Input('yaxis-column', 'value'),
    Input('xaxis-type', 'value'),
    Input('yaxis-type', 'value'),
    Input('year--slider', 'value'))
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]

    fig = px.scatter(x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
                     y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
                     hover_name=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'])

    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

    fig.update_xaxes(title=xaxis_column_name,
                     type='linear' if xaxis_type == 'Linear' else 'log')

    fig.update_yaxes(title=yaxis_column_name,
                     type='linear' if yaxis_type == 'Linear' else 'log')

    return fig


if __name__ == '__main__':
    app.run_server( mode='inline', host="localhost",port=8051)

In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

import pandas as pd

from jupyter_dash import JupyterDash

In [21]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    dcc.Input(
        id='num-multi',
        type='number',
        value=5
    ),
    html.Table([
        html.Tr([html.Td(['x', html.Sup(2)]), html.Td(id='square')]),
        html.Tr([html.Td(['x', html.Sup(3)]), html.Td(id='cube')]),
        html.Tr([html.Td([2, html.Sup('x')]), html.Td(id='twos')]),
        html.Tr([html.Td([3, html.Sup('x')]), html.Td(id='threes')]),
        html.Tr([html.Td(['x', html.Sup('x')]), html.Td(id='x^x')]),
    ]),
])


@app.callback(
    Output('square', 'children'),
    Output('cube', 'children'),
    Output('twos', 'children'),
    Output('threes', 'children'),
    Output('x^x', 'children'),
    Input('num-multi', 'value'))
def callback_a(x):
    return x**2, x**3, 2**x, 3**x, x**x


if __name__ == '__main__':
    app.run_server(mode='inline', host="localhost",port=8051)

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
~\AppData\Roaming\Python\Python39\site-packages\dash\dash.py in dispatch(
    self=<jupyter_dash.jupyter_app.JupyterDash object>
)
   1421         try:
-> 1422             cb = self.callback_map[output]
        cb = undefined
        self.callback_map = {'..square.children...cube.children...twos.children...threes.children...x^x.children..': {'inputs': [{'id': 'num-multi', 'property': 'value'}], 'state': [], 'outputs_indices': [0, 1, 2, 3, 4], 'inputs_state_indices': 0, 'callback': <function callback_a at 0x0000015F1BC4DC10>}}
        output = 'x-time-series.figure'
   1423             func = cb["callback"]

KeyError: 'x-time-series.figure'

The above exception was the direct cause of the following exception:

KeyError                                  Traceback (most recent call last)
C:\Users\anaconda3\lib\site-packages\flask\app.py in 

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
~\AppData\Roaming\Python\Python39\site-packages\dash\dash.py in dispatch(
    self=<jupyter_dash.jupyter_app.JupyterDash object>
)
   1421         try:
-> 1422             cb = self.callback_map[output]
        cb = undefined
        self.callback_map = {'..square.children...cube.children...twos.children...threes.children...x^x.children..': {'inputs': [{'id': 'num-multi', 'property': 'value'}], 'state': [], 'outputs_indices': [0, 1, 2, 3, 4], 'inputs_state_indices': 0, 'callback': <function callback_a at 0x0000015F1BC4DC10>}}
        output = 'y-time-series.figure'
   1423             func = cb["callback"]

KeyError: 'y-time-series.figure'

The above exception was the direct cause of the following exception:

KeyError                                  Traceback (most recent call last)
C:\Users\anaconda3\lib\site-packages\flask\app.py in 

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
~\AppData\Roaming\Python\Python39\site-packages\dash\dash.py in dispatch(
    self=<jupyter_dash.jupyter_app.JupyterDash object>
)
   1421         try:
-> 1422             cb = self.callback_map[output]
        cb = undefined
        self.callback_map = {'..square.children...cube.children...twos.children...threes.children...x^x.children..': {'inputs': [{'id': 'num-multi', 'property': 'value'}], 'state': [], 'outputs_indices': [0, 1, 2, 3, 4], 'inputs_state_indices': 0, 'callback': <function callback_a at 0x0000015F1BC4DC10>}}
        output = 'x-time-series.figure'
   1423             func = cb["callback"]

KeyError: 'x-time-series.figure'

The above exception was the direct cause of the following exception:

KeyError                                  Traceback (most recent call last)
C:\Users\anaconda3\lib\site-packages\flask\app.py in 

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
~\AppData\Roaming\Python\Python39\site-packages\dash\dash.py in dispatch(
    self=<jupyter_dash.jupyter_app.JupyterDash object>
)
   1421         try:
-> 1422             cb = self.callback_map[output]
        cb = undefined
        self.callback_map = {'..square.children...cube.children...twos.children...threes.children...x^x.children..': {'inputs': [{'id': 'num-multi', 'property': 'value'}], 'state': [], 'outputs_indices': [0, 1, 2, 3, 4], 'inputs_state_indices': 0, 'callback': <function callback_a at 0x0000015F1BC4DC10>}}
        output = 'x-time-series.figure'
   1423             func = cb["callback"]

KeyError: 'x-time-series.figure'

The above exception was the direct cause of the following exception:

KeyError                                  Traceback (most recent call last)
C:\Users\anaconda3\lib\site-packages\flask\app.py in 

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
~\AppData\Roaming\Python\Python39\site-packages\dash\dash.py in dispatch(
    self=<jupyter_dash.jupyter_app.JupyterDash object>
)
   1421         try:
-> 1422             cb = self.callback_map[output]
        cb = undefined
        self.callback_map = {'..square.children...cube.children...twos.children...threes.children...x^x.children..': {'inputs': [{'id': 'num-multi', 'property': 'value'}], 'state': [], 'outputs_indices': [0, 1, 2, 3, 4], 'inputs_state_indices': 0, 'callback': <function callback_a at 0x0000015F1BC4DC10>}}
        output = 'x-time-series.figure'
   1423             func = cb["callback"]

KeyError: 'x-time-series.figure'

The above exception was the direct cause of the following exception:

KeyError                                  Traceback (most recent call last)
C:\Users\anaconda3\lib\site-packages\flask\app.py in 

...

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
~\AppData\Roaming\Python\Python39\site-packages\dash\dash.py in dispatch(
    self=<jupyter_dash.jupyter_app.JupyterDash object>
)
   1421         try:
-> 1422             cb = self.callback_map[output]
        cb = undefined
        self.callback_map = {'..square.children...cube.children...twos.children...threes.children...x^x.children..': {'inputs': [{'id': 'num-multi', 'property': 'value'}], 'state': [], 'outputs_indices': [0, 1, 2, 3, 4], 'inputs_state_indices': 0, 'callback': <function callback_a at 0x0000015F1BC4DC10>}}
        output = 'x-time-series.figure'
   1423             func = cb["callback"]

KeyError: 'x-time-series.figure'

The above exception was the direct cause of the following exception:

KeyError                                  Traceback (most recent call last)
C:\Users\anaconda3\lib\site-packages\flask\app.py in 

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
~\AppData\Roaming\Python\Python39\site-packages\dash\dash.py in dispatch(
    self=<jupyter_dash.jupyter_app.JupyterDash object>
)
   1421         try:
-> 1422             cb = self.callback_map[output]
        cb = undefined
        self.callback_map = {'..square.children...cube.children...twos.children...threes.children...x^x.children..': {'inputs': [{'id': 'num-multi', 'property': 'value'}], 'state': [], 'outputs_indices': [0, 1, 2, 3, 4], 'inputs_state_indices': 0, 'callback': <function callback_a at 0x0000015F1BC4DC10>}}
        output = 'y-time-series.figure'
   1423             func = cb["callback"]

KeyError: 'y-time-series.figure'

The above exception was the direct cause of the following exception:

KeyError                                  Traceback (most recent call last)
C:\Users\anaconda3\lib\site-packages\flask\app.py in 

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
~\AppData\Roaming\Python\Python39\site-packages\dash\dash.py in dispatch(
    self=<jupyter_dash.jupyter_app.JupyterDash object>
)
   1421         try:
-> 1422             cb = self.callback_map[output]
        cb = undefined
        self.callback_map = {'..square.children...cube.children...twos.children...threes.children...x^x.children..': {'inputs': [{'id': 'num-multi', 'property': 'value'}], 'state': [], 'outputs_indices': [0, 1, 2, 3, 4], 'inputs_state_indices': 0, 'callback': <function callback_a at 0x0000015F1BC4DC10>}}
        output = 'y-time-series.figure'
   1423             func = cb["callback"]

KeyError: 'y-time-series.figure'

The above exception was the direct cause of the following exception:

KeyError                                  Traceback (most recent call last)
C:\Users\anaconda3\lib\site-packages\flask\app.py in 

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
~\AppData\Roaming\Python\Python39\site-packages\dash\dash.py in dispatch(
    self=<jupyter_dash.jupyter_app.JupyterDash object>
)
   1421         try:
-> 1422             cb = self.callback_map[output]
        cb = undefined
        self.callback_map = {'..square.children...cube.children...twos.children...threes.children...x^x.children..': {'inputs': [{'id': 'num-multi', 'property': 'value'}], 'state': [], 'outputs_indices': [0, 1, 2, 3, 4], 'inputs_state_indices': 0, 'callback': <function callback_a at 0x0000015F1BC4DC10>}}
        output = 'x-time-series.figure'
   1423             func = cb["callback"]

KeyError: 'x-time-series.figure'

The above exception was the direct cause of the following exception:

KeyError                                  Traceback (most recent call last)
C:\Users\anaconda3\lib\site-packages\flask\app.py in 

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
~\AppData\Roaming\Python\Python39\site-packages\dash\dash.py in dispatch(
    self=<jupyter_dash.jupyter_app.JupyterDash object>
)
   1421         try:
-> 1422             cb = self.callback_map[output]
        cb = undefined
        self.callback_map = {'..square.children...cube.children...twos.children...threes.children...x^x.children..': {'inputs': [{'id': 'num-multi', 'property': 'value'}], 'state': [], 'outputs_indices': [0, 1, 2, 3, 4], 'inputs_state_indices': 0, 'callback': <function callback_a at 0x0000015F1BC4DC10>}}
        output = 'x-time-series.figure'
   1423             func = cb["callback"]

KeyError: 'x-time-series.figure'

The above exception was the direct cause of the following exception:

KeyError                                  Traceback (most recent call last)
C:\Users\anaconda3\lib\site-packages\flask\app.py in 

In [23]:
from dash import Dash, html, dcc, Input, Output
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')


app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Fertility rate, total (births per woman)',
                id='crossfilter-xaxis-column',
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='crossfilter-xaxis-type',
                labelStyle={'display': 'inline-block', 'marginTop': '5px'}
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Life expectancy at birth, total (years)',
                id='crossfilter-yaxis-column'
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='crossfilter-yaxis-type',
                labelStyle={'display': 'inline-block', 'marginTop': '5px'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'padding': '10px 5px'
    }),

    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': 'Japan'}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),

    html.Div(dcc.Slider(
        df['Year'].min(),
        df['Year'].max(),
        step=None,
        id='crossfilter-year--slider',
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()}
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])


@app.callback(
    Output('crossfilter-indicator-scatter', 'figure'),
    Input('crossfilter-xaxis-column', 'value'),
    Input('crossfilter-yaxis-column', 'value'),
    Input('crossfilter-xaxis-type', 'value'),
    Input('crossfilter-yaxis-type', 'value'),
    Input('crossfilter-year--slider', 'value'))
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]

    fig = px.scatter(x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
            y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
            hover_name=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name']
            )

    fig.update_traces(customdata=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'])

    fig.update_xaxes(title=xaxis_column_name, type='linear' if xaxis_type == 'Linear' else 'log')

    fig.update_yaxes(title=yaxis_column_name, type='linear' if yaxis_type == 'Linear' else 'log')

    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

    return fig


def create_time_series(dff, axis_type, title):

    fig = px.scatter(dff, x='Year', y='Value')

    fig.update_traces(mode='lines+markers')

    fig.update_xaxes(showgrid=False)

    fig.update_yaxes(type='linear' if axis_type == 'Linear' else 'log')

    fig.add_annotation(x=0, y=0.85, xanchor='left', yanchor='bottom',
                       xref='paper', yref='paper', showarrow=False, align='left',
                       text=title)

    fig.update_layout(height=225, margin={'l': 20, 'b': 30, 'r': 10, 't': 10})

    return fig


@app.callback(
    Output('x-time-series', 'figure'),
    Input('crossfilter-indicator-scatter', 'hoverData'),
    Input('crossfilter-xaxis-column', 'value'),
    Input('crossfilter-xaxis-type', 'value'))
def update_y_timeseries(hoverData, xaxis_column_name, axis_type):
    country_name = hoverData['points'][0]['customdata']
    dff = df[df['Country Name'] == country_name]
    dff = dff[dff['Indicator Name'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)


@app.callback(
    Output('y-time-series', 'figure'),
    Input('crossfilter-indicator-scatter', 'hoverData'),
    Input('crossfilter-yaxis-column', 'value'),
    Input('crossfilter-yaxis-type', 'value'))
def update_x_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['Country Name'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['Indicator Name'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)


if __name__ == '__main__':
    app.run_server(mode='inline', host="localhost",port=8051)

In [6]:
from dash import Dash, html, dcc
from jupyter_dash import JupyterDash

app = JupyterDash(__name__)




app.layout = html.Div([
    dcc.Dropdown(['New York City', 'Montréal', 'San Francisco'], 'Montréal')
])




if __name__ == '__main__':
    app.run_server(mode='inline', host="localhost",port=8051)

In [ ]:
app.layout = html.Div()

In [ ]:
[dcc.Dropdown(['New York City', 'Montréal', 'San Francisco'], 'Montréal')]

In [9]:
from dash import Dash, html, dcc
from jupyter_dash import JupyterDash

app = JupyterDash(__name__)




app.layout = html.Div([
    dcc.Dropdown(['New York City', 'Montréal', 'San Francisco'], 'Montréal', multi = True)
])




if __name__ == '__main__':
    app.run_server(mode='inline', host="localhost",port=8051)

In [2]:
from dash import Dash, html, dcc
from jupyter_dash import JupyterDash

app = JupyterDash(__name__)




app.layout = html.Div([
    dcc.Dropdown(['New York City', 'Montréal', 'San Francisco'], 'Montréal')
])




if __name__ == '__main__':
    app.run_server(mode='inline', host="localhost",port=8051)

In [16]:
from dash import dcc
import plotly.express as px

df = px.data.iris()  # iris is a pandas DataFrame
fig = px.scatter(df, x="sepal_width", y="sepal_length")
fig.show()

dcc.Graph(figure=fig);

In [17]:
from dash import dcc
import plotly.graph_objs as go

fig = go.Figure(data=[go.Scatter(x=[1, 2, 3], y=[4, 1, 2])])
fig.show()
dcc.Graph(figure=fig)

Graph(figure=Figure({
    'data': [{'type': 'scatter', 'x': [1, 2, 3], 'y': [4, 1, 2]}], 'layout': {'template': '...'}
}))